In [2]:
!pip3 install libtiff

    100% |████████████████████████████████| 133kB 342kB/s ta 0:00:01
  Running setup.py bdist_wheel for libtiff ... done
  Stored in directory: /root/.cache/pip/wheels/7c/ce/79/9c7115224f798f73bdbd2c23e06c6fa048adcca7041b9fd104
Successfully built libtiff
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
# coding:utf-8
# import gdal
import numpy as np
import os
from libtiff import TIFF

unit_width = 224  # 每个小图片的长和宽（正方形）
file_dir = '/input0/yuanyang_land/data'


def readTif(file_name, isMask=False):
    tif = TIFF.open(file_dir + file_name)

    image = tif.read_image()
    im_width = image.shape[0]
    im_height = image.shape[1]
    im_bands = 1
    if not isMask:
        im_width = image.shape[0]
        im_height = image.shape[1]
        im_bands = image.shape[2]
        image = image.swapaxes(0, 2)
        im_data = image.swapaxes(1, 2)
        image = im_data

    return {'width': im_width, 'height': im_height, 'band_num': im_bands, 'data': image}


def divide(img, mask, size, stride):
    imgs, labels = [], []
    bands, height, width = img['band_num'], img['height'], img['width']
    for row in range(0, height, stride):
        for col in range(0, width, stride):
            if row + size > height or col + size > width:
                continue
            feature = []
            for band_id in range(bands):
                feature.append(img['data'][band_id, row:row + size, col:col + size])
            label = mask['data'][row:row + size, col: col + size]
            imgs.append(feature)
            labels.append(label)
    return np.array(imgs), labels


def get_data():
    img_names = ['/images/%02d_4bands.tif' % i for i in range(1, 70)]
    mask_names = ['/labels/%02d_4bands_mask.tif' % i for i in range(1, 70)]
    print(img_names)
    xs, ys = [], []
    for (img, mask) in zip(img_names, mask_names):
        img = readTif(img)
        mask = readTif(mask, True)
        imgs, labels = divide(img, mask, 224, 100)  # 后两个参数为图片尺寸和步长
        #         print('2', imgs.shape)
        for img, label in zip(imgs, labels):
            xs.append(img)
            ys.append(label)
    return xs, ys


xs, ys = get_data()

ys = np.array(ys)
orishape = ys.shape
ys = ys.reshape(-1)
unique_num = {v: i for i, v in enumerate(np.unique(ys))}
for i, v in enumerate(ys):
    ys[i] = unique_num[v]
ys = ys.reshape(orishape)
xs = np.asarray(xs)
ys = np.asarray(ys)
pi = np.random.permutation(xs.shape[0])
xs, ys = xs[pi], ys[pi]
bound = int(len(ys) * 0.8)

new_mask = np.zeros(ys.shape + (2,))
for i in range(2):
    new_mask[ys == i, i] = 1
ys = new_mask.astype('int32')

if not os.path.exists('./data'):
    os.mkdir('./data')

np.save('./data/train_image.npy', xs[:bound])
np.save('./data/train_annotation.npy', ys[:bound])
np.save('./data/validation_image.npy', xs[bound:])
np.save('./data/validation_annotation.npy', ys[bound:])

e_mask = np.load('./data/demo_data/train_annotation.npy')
print('e_mask.shape', e_mask.shape)

['/images/01_4bands.tif', '/images/02_4bands.tif', '/images/03_4bands.tif', '/images/04_4bands.tif', '/images/05_4bands.tif', '/images/06_4bands.tif', '/images/07_4bands.tif', '/images/08_4bands.tif', '/images/09_4bands.tif', '/images/10_4bands.tif', '/images/11_4bands.tif', '/images/12_4bands.tif', '/images/13_4bands.tif', '/images/14_4bands.tif', '/images/15_4bands.tif', '/images/16_4bands.tif', '/images/17_4bands.tif', '/images/18_4bands.tif', '/images/19_4bands.tif', '/images/20_4bands.tif', '/images/21_4bands.tif', '/images/22_4bands.tif', '/images/23_4bands.tif', '/images/24_4bands.tif', '/images/25_4bands.tif', '/images/26_4bands.tif', '/images/27_4bands.tif', '/images/28_4bands.tif', '/images/29_4bands.tif', '/images/30_4bands.tif', '/images/31_4bands.tif', '/images/32_4bands.tif', '/images/33_4bands.tif', '/images/34_4bands.tif', '/images/35_4bands.tif', '/images/36_4bands.tif', '/images/37_4bands.tif', '/images/38_4bands.tif', '/images/39_4bands.tif', '/images/40_4bands.tif',